In [171]:
using Glob
using Dates
using DIVAnd
using NCDatasets
using GeoDatasets
using CairoMakie, GeoMakie
using JupyterFormatter
enable_autoformat()
include("./config.jl")
monthlist = [Dates.monthname(mm) for mm = 1:12];

In [173]:
for varname in varlist
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    obsyears = Int64[]
    obsmonths = Int64[]

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsvalue, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)
        append!(obsyears, Dates.year.(obstime))
        append!(obsmonths, Dates.month.(obstime))
    end

    @info(extrema(obsyears))
    yearmin = minimum(obsyears)
    yearmax = maximum(obsyears)
    nobs_month = [length(findall(obsmonths .== mm)) for mm = 1:12]
    nobs_year = [length(findall(obsyears .== yyyy)) for yyyy = yearmin:yearmax]

    # Make figure
    fig1 = make_histogram(yearmin, yearmax, nobs_year, nobs_month, varname_)
    save(joinpath(figdir, "histogram_$varname.png"), fig1)
end

[ Info: (1960, 2023)
[ Info: (1963, 2024)
[ Info: (1960, 2023)
[ Info: (1950, 2023)
[ Info: (1928, 2023)
[ Info: (1902, 2024)


## Make figure

make_histogram (generic function with 1 method)